In [1]:
import pandas as pd
import numpy as np
import itertools
import igraph
import sys
import collections
import datetime
import random

In [2]:
G = igraph.Graph.Read_GraphML(f = 'networks/EM-TE-16.graphml')

In [3]:
G_orig = G.copy()

In [4]:
stop_times_od = pd.read_csv('networks/EM-TE-16.csv')

In [5]:
stop_times_od.head()

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id,node_o,node_d,link
0,32828690,28,31,6,0,13464667.0,32,6,1,13464586.0,6:00,6:01,06:00:00,06:01:00,1,506,0,13464667-506-6-0,13464586-506-6-1,0-506-32828690-0_13464667-506-6-0_13464586-506...
1,32828690,29,32,6,1,13464586.0,33,6,1,13464490.0,6:01,6:01,06:01:00,06:01:00,0,506,0,13464586-506-6-1,13464490-506-6-1,0-506-32828690-0_13464586-506-6-1_13464490-506...
2,32828690,30,33,6,1,13464490.0,34,6,2,13464269.0,6:01,6:02,06:01:00,06:02:00,1,506,0,13464490-506-6-1,13464269-506-6-2,0-506-32828690-0_13464490-506-6-1_13464269-506...
3,32828690,31,34,6,2,13464269.0,35,6,3,13464177.0,6:02,6:03,06:02:00,06:03:00,1,506,0,13464269-506-6-2,13464177-506-6-3,0-506-32828690-0_13464269-506-6-2_13464177-506...
4,32828690,32,35,6,3,13464177.0,36,6,4,13464004.0,6:03,6:04,06:03:00,06:04:00,1,506,0,13464177-506-6-3,13464004-506-6-4,0-506-32828690-0_13464177-506-6-3_13464004-506...


In [6]:
od_data = pd.read_csv('TTS OD/LICO_TTS.csv')
od_data['origin'] = od_data['origin'] + 1000
od_data['destination'] = od_data['destination'] + 1000
od_data

,origin,destination,type,total
0,1001,1018,raw_am,15
1,1001,1035,raw_am,14
2,1001,1057,raw_am,20
3,1001,1068,raw_am,14
4,1001,1212,raw_am,8
...,...,...,...,...
30463,1603,1223,c0_ev,8
30464,1610,1561,c0_ev,22
30465,1613,1523,c0_ev,21
30466,1613,1578,c0_ev,47


In [7]:
depart_hour = '-4-'
rand_time = []

for i in range(6):
    minute = random.randint(0,9)
    minute = minute + i*10
    rand_time.append(depart_hour + str(minute))

In [8]:
node_key_table = []

for i in G.vs:
    node_key_table.append([i.index,  i['id']])
node_df = pd.DataFrame.from_records(node_key_table, columns = ['index', 'node_id'])

In [9]:
node_df['INT_ID'] = node_df['node_id'].str.split('-', expand = True)[0]

In [10]:
node_df['node_time'] = node_df['node_id'].str.split('-', expand = True)[0] + '-' + node_df['node_id'].str.split('-', expand = True)[2] + '-' + node_df['node_id'].str.split('-', expand = True)[3]

In [11]:
node_dict = node_df[['INT_ID']].to_dict()['INT_ID']

In [12]:
time_dict = node_df[['node_time']].to_dict()['node_time']

In [13]:
node_df

,index,node_id,INT_ID,node_time
0,0,13464667-0-4-0,13464667,13464667-4-0
1,1,13464667-0-4-1,13464667,13464667-4-1
2,2,13464667-0-4-2,13464667,13464667-4-2
3,3,13464667-0-4-3,13464667,13464667-4-3
4,4,13464667-0-4-4,13464667,13464667-4-4
...,...,...,...,...
1472384,1472384,13452625-0-8-55,13452625,13452625-8-55
1472385,1472385,13452625-0-8-56,13452625,13452625-8-56
1472386,1472386,13452625-0-8-57,13452625,13452625-8-57
1472387,1472387,13452625-0-8-58,13452625,13452625-8-58


## One Iteration of BC Calculation

### Node

In [14]:
node_data = od_data[od_data['type'] == 'raw_em'][['origin', 'destination']].copy()
node_data['origin'] = node_data['origin'].astype(str)
node_data['destination'] = node_data['destination'].astype(str)
node_data['o_node'] = node_data['origin'] + '-0' + rand_time[0]
node_data['d_node'] = node_data['destination'] + '-0-99-99'
node_data

,origin,destination,o_node,d_node
226,1018,1034,1018-0-4-2,1034-0-99-99
462,1021,1361,1021-0-4-2,1361-0-99-99
465,1021,1386,1021-0-4-2,1386-0-99-99
502,1021,1519,1021-0-4-2,1519-0-99-99
509,1021,1545,1021-0-4-2,1545-0-99-99
...,...,...,...,...
23621,1606,1068,1606-0-4-2,1068-0-99-99
23636,1606,1415,1606-0-4-2,1415-0-99-99
23822,1619,1056,1619-0-4-2,1056-0-99-99
23855,1620,1220,1620-0-4-2,1220-0-99-99


In [15]:
origin = node_data.loc[462,'o_node']
dest = node_data.loc[462,'d_node']
origin, dest

('1021-0-4-2', '1361-0-99-99')

In [16]:
out = G.get_all_shortest_paths(G.vs.select(id_eq = origin)[0], G.vs.select(id_eq = dest)[0], weights = 'cost',  mode = 'out')
len(out)

41234

In [17]:
table = []
time_table = []
for path in out:
    temp_list = list(set(map(node_dict.get, path)))
    time_list = list(set(map(time_dict.get, path)))
    table.append(temp_list)
    time_table.append(time_list)

In [18]:
time_table

[['13463124-4-12',
  '13469317-6-10',
  '13466999-5-3',
  '13463124-4-14',
  '200-5-51',
  '13466999-5-24',
  '200-5-55',
  '13466999-5-20',
  '208-4-36',
  '203-4-42',
  '13466999-4-58',
  '13468811-6-2',
  '13469285-6-4',
  '13463124-4-10',
  '13466999-4-51',
  '212-4-28',
  '106-4-20',
  '13463124-4-17',
  '13466999-5-11',
  '13466999-5-13',
  '13466999-5-23',
  '13466999-4-47',
  '156-4-23',
  '200-6-0',
  '13466999-4-44',
  '13465705-4-35',
  '201-5-34',
  '200-5-42',
  '204-4-44',
  '13469211-6-4',
  '200-5-48',
  '13468974-6-3',
  '13469317-6-11',
  '200-5-56',
  '207-4-37',
  '13466999-4-57',
  '214-4-25',
  '200-5-57',
  '13469317-6-12',
  '202-5-32',
  '13466999-5-12',
  '13463747-4-22',
  '13466999-5-10',
  '13469317-6-9',
  '200-5-38',
  '13466999-4-55',
  '200-5-47',
  '13466999-5-25',
  '200-5-40',
  '200-5-44',
  '13466999-5-27',
  '13463124-4-13',
  '13466999-4-59',
  '205-4-40',
  '1361-99-99',
  '1021-4-2',
  '13466999-5-15',
  '13466999-4-56',
  '13469317-6-7',
  '20

In [19]:

table.sort()
len(list(table for table,_ in itertools.groupby(table)))

58

In [20]:
node_path = list(itertools.chain.from_iterable(table))
time_path = list(itertools.chain.from_iterable(time_table))

In [21]:
single_path = collections.Counter(node_path)
single_time_path = collections.Counter(time_path)

In [22]:
single_path

Counter({'13454935': 41234,
         '1021': 41234,
         '106': 41234,
         '13469285': 13630,
         '13468683': 13630,
         '13468811': 13630,
         '215': 41234,
         '13466999': 41234,
         '208': 41234,
         '210': 41234,
         '200': 41228,
         '156': 41234,
         '13465705': 41234,
         '13468685': 450,
         '213': 41234,
         '211': 41234,
         '13463124': 20617,
         '209': 41234,
         '201': 41234,
         '205': 41234,
         '13469317': 41234,
         '202': 41234,
         '13469211': 13630,
         '1361': 41234,
         '13468974': 13630,
         '203': 41234,
         '216': 41234,
         '206': 41234,
         '207': 41234,
         '214': 41234,
         '13467917': 750,
         '13463747': 41234,
         '212': 41234,
         '13467231': 41234,
         '13468117': 600,
         '13467131': 41234,
         '13465043': 41234,
         '204': 41234,
         '13469002': 300,
         '13469132'

In [23]:
single_path = {k: v / len(out) for k, v in dict(single_path).items()}
single_time_path = {k: v / len(out) for k, v in dict(single_time_path).items()}

In [24]:
pd.DataFrame(single_path.items(), columns = ['INT_ID', 'bc_single'])

,INT_ID,bc_single
0,13454935,1.000000
1,1021,1.000000
2,106,1.000000
3,13469285,0.330552
4,13468683,0.330552
5,13468811,0.330552
6,215,1.000000
7,13466999,1.000000
8,208,1.000000
9,210,1.000000


In [25]:
pd.DataFrame(single_time_path.items(), columns = ['node_id', 'tv_bc_single'])

,node_id,tv_bc_single
0,13463124-4-12,0.500000
1,13469317-6-10,0.198331
2,13466999-5-3,0.047243
3,13463124-4-14,0.500000
4,200-5-51,0.081486
...,...,...
881,13468202-5-42,0.000049
882,13468202-5-21,0.000049
883,13468202-5-48,0.000049
884,13468202-5-50,0.000049


### Edge

In [27]:
edge_list = []
for path in out:
    flag_start = 0
    for node in path:
        if flag_start == 0:
            flag_start = flag_start + 1
            from_node = node
            continue
        to_node = node
        edge_list.append([origin, dest, from_node, to_node])
        from_node = to_node

In [29]:
total_paths = len(out)
total_paths

41234

In [40]:
edge_df = pd.DataFrame.from_records(edge_list, columns = ['o_node_time', 'd_node_time', 'index_from', 'index_to'])

edge_df = edge_df.merge(node_df, left_on = ['index_from'], right_on = ['index'])
edge_df = edge_df.rename(columns = {'INT_ID':'INT_ID_o', 'node_id':'node_o'})
edge_df = edge_df.drop(columns = ['index'])
edge_df = edge_df.merge(node_df, left_on = ['index_to'], right_on = ['index'])
edge_df = edge_df.rename(columns = {'INT_ID':'INT_ID_d', 'node_id':'node_d'})
edge_df['route_o'] = edge_df['node_o'].str.split('-', expand = True)[1]
edge_df['route_d'] = edge_df['node_d'].str.split('-', expand = True)[1]

edge_df['route'] = np.where(edge_df['route_o'] == edge_df['route_d'], edge_df['route_o'], np.nan)
edge_df['route'] = edge_df['route'].astype(float)
edge_df['route'] = np.where(edge_df['route']!=0, edge_df['route'], np.nan)

edge_df = edge_df.drop(columns = ['index', 'route_o', 'route_d'])
edge_df

,o_node_time,d_node_time,index_from,index_to,node_o,INT_ID_o,node_time_x,node_d,INT_ID_d,node_time_y,route
0,1021-0-4-2,1361-0-99-99,1284866,515710,1021-0-4-2,1021,1021-4-2,13463124-0-4-10,13463124,13463124-4-10,NaN
1,1021-0-4-2,1361-0-99-99,1284866,515710,1021-0-4-2,1021,1021-4-2,13463124-0-4-10,13463124,13463124-4-10,NaN
2,1021-0-4-2,1361-0-99-99,1284866,515710,1021-0-4-2,1021,1021-4-2,13463124-0-4-10,13463124,13463124-4-10,NaN
3,1021-0-4-2,1361-0-99-99,1284866,515710,1021-0-4-2,1021,1021-4-2,13463124-0-4-10,13463124,13463124-4-10,NaN
4,1021-0-4-2,1361-0-99-99,1284866,515710,1021-0-4-2,1021,1021-4-2,13463124-0-4-10,13463124,13463124-4-10,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4320032,1021-0-4-2,1361-0-99-99,145316,145317,13468202-0-5-56,13468202,13468202-5-56,13468202-0-5-57,13468202,13468202-5-57,NaN
4320033,1021-0-4-2,1361-0-99-99,145317,145318,13468202-0-5-57,13468202,13468202-5-57,13468202-0-5-58,13468202,13468202-5-58,NaN
4320034,1021-0-4-2,1361-0-99-99,145317,145318,13468202-0-5-57,13468202,13468202-5-57,13468202-0-5-58,13468202,13468202-5-58,NaN
4320035,1021-0-4-2,1361-0-99-99,145318,1077191,13468202-0-5-58,13468202,13468202-5-58,13468202-112-6-0,13468202,13468202-6-0,NaN


In [41]:
edge_df = edge_df[['o_node_time', 'd_node_time', 'INT_ID_o', 'INT_ID_d', 'route']]
edge_df['bc_single'] = True
edge_df = edge_df.groupby(['o_node_time', 'd_node_time', 'INT_ID_o', 'INT_ID_d', 'route']).count().reset_index()

<ipython-input-41-f2996628da3b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_df['bc_single'] = True


In [42]:
edge_df['bc_single'] = edge_df['bc_single']/len(out)
edge_df

,o_node_time,d_node_time,INT_ID_o,INT_ID_d,route,bc_single
0,1021-0-4-2,1361-0-99-99,106,215,300.0,1.0
1,1021-0-4-2,1361-0-99-99,13463124,216,300.0,0.5
2,1021-0-4-2,1361-0-99-99,13463747,156,300.0,1.0
3,1021-0-4-2,1361-0-99-99,13465043,210,300.0,1.0
4,1021-0-4-2,1361-0-99-99,13465705,208,300.0,1.0
...,...,...,...,...,...,...
78,1021-0-4-2,1361-0-99-99,212,211,300.0,1.0
79,1021-0-4-2,1361-0-99-99,213,212,300.0,1.0
80,1021-0-4-2,1361-0-99-99,214,213,300.0,1.0
81,1021-0-4-2,1361-0-99-99,215,13463747,300.0,1.0


## Time Period 0

In [46]:
od_list = []
for period in range(6):
    
    node_data = od_data[od_data['type'] == 'raw_em'][['origin', 'destination']].copy()
    node_data['origin'] = node_data['origin'].astype(str)
    node_data['destination'] = node_data['destination'].astype(str)
    node_data['o_node'] = node_data['origin'] + '-0' + rand_time[period]
    node_data['d_node'] = node_data['destination'] + '-0-99-99'

    od_list.append(node_data[['o_node', 'd_node']].values.tolist())


In [47]:
od_list = list(itertools.chain(*od_list))

In [48]:
len(od_list)

1056

In [49]:
j = 0

iteration_start = datetime.datetime.now()
    
for pair in od_list:

    o_node = pair[0]
    d_node = pair[1]
    
    out = G.get_all_shortest_paths(G.vs.select(id_eq = pair[0])[0], 
                                   G.vs.select(id_eq = pair[1])[0], weights = 'cost',  mode = 'out')
    
    table = []
    time_table = []
    for path in out:
        temp_list = list(set(map(node_dict.get, path)))
        time_list = list(set(map(time_dict.get, path)))
        table.append(temp_list)
        time_table.append(time_list)

    node_path = list(itertools.chain.from_iterable(table))
    time_path = list(itertools.chain.from_iterable(time_table))

    single_path = collections.Counter(node_path)
    single_time_path = collections.Counter(time_path)

    single_path = {k: v / len(out) for k, v in dict(single_path).items()}
    single_time_path = {k: v / len(out) for k, v in dict(single_time_path).items()}

    single_df = pd.DataFrame(single_path.items(), columns = ['INT_ID', 'bc_single'])
    single_time_df = pd.DataFrame(single_time_path.items(), columns = ['node_id', 'tv_bc_single'])

    single_df['o_node_time'] = o_node
    single_df['d_node_time'] = d_node

    single_time_df['o_node_time'] = o_node
    single_time_df['d_node_time'] = d_node
    
    # edge BC
    edge_list = []
    for path in out:
        flag_start = 0
        for node in path:
            if flag_start == 0:
                flag_start = flag_start + 1
                from_node = node
                continue
            to_node = node
            edge_list.append([o_node, d_node, from_node, to_node])
            from_node = to_node
    total_paths = len(out)
    edge_df = pd.DataFrame.from_records(edge_list, columns = ['o_node_time', 'd_node_time', 'index_from', 'index_to'])

    edge_df = edge_df.merge(node_df, left_on = ['index_from'], right_on = ['index'])
    edge_df = edge_df.rename(columns = {'INT_ID':'INT_ID_o', 'node_id':'node_o'})
    edge_df = edge_df.drop(columns = ['index'])
    edge_df = edge_df.merge(node_df, left_on = ['index_to'], right_on = ['index'])
    edge_df = edge_df.rename(columns = {'INT_ID':'INT_ID_d', 'node_id':'node_d'})
    edge_df['route_o'] = edge_df['node_o'].str.split('-', expand = True)[1]
    edge_df['route_d'] = edge_df['node_d'].str.split('-', expand = True)[1]

    edge_df['route'] = np.where(edge_df['route_o'] == edge_df['route_d'], edge_df['route_o'], np.nan)
    edge_df['route'] = edge_df['route'].astype(float)
    edge_df['route'] = np.where(edge_df['route']!=0, edge_df['route'], np.nan)

    edge_df = edge_df.drop(columns = ['index', 'route_o', 'route_d'])
    
    edge_df = edge_df[['o_node_time', 'd_node_time', 'INT_ID_o', 'INT_ID_d', 'route']]
    edge_df['bc_single'] = True
    edge_df = edge_df.groupby(['o_node_time', 'd_node_time', 'INT_ID_o', 'INT_ID_d', 'route']).count().reset_index()
    edge_df['bc_single'] = edge_df['bc_single']/len(out)

    if j == 0:
        bc_df = single_df 
        tvbc_df = single_time_df
        edge_df_all = edge_df
    else:
        bc_df = bc_df.append(single_df)
        tvbc_df = tvbc_df.append(single_time_df)
        edge_df_all = edge_df_all.append(edge_df) 

    j = j + 1
    
    if j%50 == 0:
        print(j)
    
end = datetime.datetime.now() - iteration_start
print(end)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
0:22:14.029770


In [50]:
bc_df['o_node'] = bc_df['o_node_time'].str.split('-', expand = True)[0]
bc_df['d_node'] = bc_df['d_node_time'].str.split('-', expand = True)[0]

In [51]:
key_table = pd.DataFrame.from_records(od_list, columns = ['o_node_time', 'd_node_time'])

key_table['o_node'] = key_table['o_node_time'].str.split('-', expand = True)[0]
key_table['d_node'] = key_table['d_node_time'].str.split('-', expand = True)[0]

key_table['key'] = 0
int_table = bc_df[['INT_ID']].drop_duplicates()
int_table['key'] = 0

In [52]:
key_table = key_table.merge(int_table, how = 'outer')

In [53]:
bc_filled = key_table.merge(bc_df, how = 'left')

In [54]:
bc_filled['bc_single'] = bc_filled['bc_single'].fillna(0)

In [55]:
bc_filled

,o_node_time,d_node_time,o_node,d_node,key,INT_ID,bc_single
0,1018-0-4-2,1034-0-99-99,1018,1034,0,13465673,1.0
1,1018-0-4-2,1034-0-99-99,1018,1034,0,13465894,1.0
2,1018-0-4-2,1034-0-99-99,1018,1034,0,1018,1.0
3,1018-0-4-2,1034-0-99-99,1018,1034,0,13465795,1.0
4,1018-0-4-2,1034-0-99-99,1018,1034,0,13466110,1.0
...,...,...,...,...,...,...,...
2347483,1621-0-4-56,1063-0-99-99,1621,1063,0,20232429,0.0
2347484,1621-0-4-56,1063-0-99-99,1621,1063,0,13444804,0.0
2347485,1621-0-4-56,1063-0-99-99,1621,1063,0,13444964,0.0
2347486,1621-0-4-56,1063-0-99-99,1621,1063,0,13445947,0.0


In [56]:
bc_filled.to_csv('networks/EM/BC-EM_node.csv', index = False)

In [57]:
bc_avg = bc_filled.groupby(['o_node', 'd_node','INT_ID']).agg({'bc_single':'mean'})

In [58]:
bc_avg = bc_avg[bc_avg['bc_single']>0]
bc_avg = bc_avg.reset_index()

In [59]:
bc_avg

,o_node,d_node,INT_ID,bc_single
0,1018,1034,1018,1.000000
1,1018,1034,1034,1.000000
2,1018,1034,13464785,0.033333
3,1018,1034,13465179,0.266667
4,1018,1034,13465567,0.300000
...,...,...,...,...
19602,1621,1063,301,0.271418
19603,1621,1063,302,0.271418
19604,1621,1063,303,0.271418
19605,1621,1063,304,0.271418


In [60]:
tvbc_df

,node_id,tv_bc_single,o_node_time,d_node_time
0,1018-4-2,1.000000,1018-0-4-2,1034-0-99-99
1,13465795-4-15,1.000000,1018-0-4-2,1034-0-99-99
2,1034-99-99,1.000000,1018-0-4-2,1034-0-99-99
3,13465673-4-12,0.500000,1018-0-4-2,1034-0-99-99
4,13465894-4-16,1.000000,1018-0-4-2,1034-0-99-99
...,...,...,...,...
791,13442126-5-47,0.036511,1621-0-4-56,1063-0-99-99
792,13442126-5-45,0.036511,1621-0-4-56,1063-0-99-99
793,13442126-5-49,0.036511,1621-0-4-56,1063-0-99-99
794,13442126-5-41,0.016227,1621-0-4-56,1063-0-99-99


In [62]:
bc_avg.to_csv('networks/EM/BC-EM_avg.csv', index = False)

In [42]:
tvbc_df['o_node'] = tvbc_df['o_node_time'].str.split('-', expand = True)[0]
tvbc_df['d_node'] = tvbc_df['d_node_time'].str.split('-', expand = True)[0]

In [43]:
tvkey_table = pd.DataFrame.from_records(od_list, columns = ['o_node_time', 'd_node_time'])

tvkey_table['o_node'] = tvkey_table['o_node_time'].str.split('-', expand = True)[0]
tvkey_table['d_node'] = tvkey_table['d_node_time'].str.split('-', expand = True)[0]

tvkey_table['key'] = 0
tvint_table = tvbc_df[['node_id']].drop_duplicates()
tvint_table['key'] = 0

In [44]:
tvkey_table = tvkey_table.merge(tvint_table, how = 'outer')

In [45]:
tvbc_filled = tvkey_table.merge(tvbc_df, how = 'left')
tvbc_filled['tv_bc_single'] = tvbc_filled['tv_bc_single'].fillna(0)

In [46]:
tvbc_filled 

,o_node_time,d_node_time,o_node,d_node,key,node_id,tv_bc_single
0,1018-0-4-9,1034-0-99-99,1018,1034,0,13464785-4-21,0.166667
1,1018-0-4-9,1034-0-99-99,1018,1034,0,14036494-4-29,1.000000
2,1018-0-4-9,1034-0-99-99,1018,1034,0,13466117-4-26,1.000000
3,1018-0-4-9,1034-0-99-99,1018,1034,0,13466475-4-28,1.000000
4,1018-0-4-9,1034-0-99-99,1018,1034,0,13465567-4-24,0.666667
...,...,...,...,...,...,...,...
35888155,1621-0-4-57,1063-0-99-99,1621,1063,0,1619-4-57,0.000000
35888156,1621-0-4-57,1063-0-99-99,1621,1063,0,101-7-17,0.000000
35888157,1621-0-4-57,1063-0-99-99,1621,1063,0,1620-4-57,0.000000
35888158,1621-0-4-57,1063-0-99-99,1621,1063,0,1621-4-57,1.000000


In [47]:
tvbc_filled.to_csv('networks/EM/BC-EM_tv-raw.csv', index = False)

In [71]:
tv_bc_avg = tvbc_filled.groupby(['o_node', 'd_node','node_id']).agg({'tv_bc_single':'mean'})

In [72]:
tv_bc_avg = tv_bc_avg[tv_bc_avg['tv_bc_single']>0]
tv_bc_avg = tv_bc_avg.reset_index()

In [63]:
edge_df_filtered = edge_df_all[~edge_df_all['route'].isna()]


In [64]:
key_links = edge_df_filtered[['INT_ID_o', 'INT_ID_d']].drop_duplicates().copy()
key_links['key'] = 0

In [65]:
key_pairs = edge_df_filtered[['o_node_time', 'd_node_time']].drop_duplicates().copy()
key_pairs['key'] = 0

In [66]:
key = key_pairs.merge(key_links, how = 'outer')
key

,o_node_time,d_node_time,key,INT_ID_o,INT_ID_d
0,1018-0-4-2,1034-0-99-99,0,13465571,13465673
1,1018-0-4-2,1034-0-99-99,0,13465673,13465795
2,1018-0-4-2,1034-0-99-99,0,13465795,13465894
3,1018-0-4-2,1034-0-99-99,0,13465894,13465979
4,1018-0-4-2,1034-0-99-99,0,13465979,13466110
...,...,...,...,...,...
3480455,1621-0-4-56,1063-0-99-99,0,13444964,13445219
3480456,1621-0-4-56,1063-0-99-99,0,13445219,13445947
3480457,1621-0-4-56,1063-0-99-99,0,13445947,13446743
3480458,1621-0-4-56,1063-0-99-99,0,13446743,303


In [67]:
key = key.merge(edge_df_filtered, how = 'left')

In [68]:
key['bc_single'] = key['bc_single'].fillna(0)

In [69]:
edge_bc_avg = key[['o_node_time', 'd_node_time', 'INT_ID_o', 'INT_ID_d', 'bc_single']].copy()

In [133]:
edge_bc_avg['o_node'] = edge_bc_avg['o_node_time'].str.split('-', expand = True)[0]
edge_bc_avg['d_node'] = edge_bc_avg['d_node_time'].str.split('-', expand = True)[0]

In [135]:
edge_bc_avg.to_csv('networks/EM/BC-EM_edge-raw.csv', index = False)

In [138]:
edge_bc_avg = edge_bc_avg.groupby(['o_node', 'd_node', 'INT_ID_o', 'INT_ID_d']).agg({'bc_single':'mean'})

In [139]:
edge_bc_avg = edge_bc_avg[edge_bc_avg['bc_single']>0]
edge_bc_avg = edge_bc_avg.reset_index()

In [153]:
edge_bc_avg.to_csv('networks/EM/BC-EM_edge-avg.csv', index = False)